In [1]:
import pandas as pd
import numpy as np 
data = pd.read_csv('training_data.csv')  ##use this data cv F1 0.50  
#data = pd.read_csv('facies_vectors.csv')  ##use this data(two more wells) cv F1 0.48 
#data.describe()
data_test = data[data['Well Name'] == 'SHANKLE']
data = data[data['Well Name'] != 'SHANKLE']
#data2 = data.sample(n=729)
#data2.describe()   # use random sampled 729 events gives 
#F1 scroe 0.51 for CV(nonstratified split), 0.36 for Test 
#F1 scroe 0.55 for CV(stratified split), 0.41 for Test 

#data['BoundaryFlag'] = pd.Series(0,index=data.index)  # add one column. 
#previousface=-1
#for index, row in data.iterrows():
#    if row['Facies']!=previousface: 
#        data.at[index, 'BoundaryFlag'] =  1   
#        data.at[index-1, 'BoundaryFlag'] =  1    
#        data.set_value(index, 'BoundaryFlag',1) ##old version
#        previousface = row['Facies']
#data = data[data['BoundaryFlag']==0]
#data= data.drop(['BoundaryFlag'],axis=1)
#data.describe
#w/o boundary in data, F1 score 0.61 for CV and 0.43 for Test 
#w boundary in data, F1 score 0.50 for CV and 0.55 for Test
data.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            3222, 3223, 3224, 3225, 3226, 3227, 3228, 3229, 3230, 3231],
           dtype='int64', length=2783)

In [2]:
features = ['GR', 'ILD_log10', 'DeltaPHI','PHIND','PE','NM_M', 'RELPOS']
feature_vectors = data[features]
facies_labels = data['Facies']

feature_vectors_test = data_test[features]
facies_labels_test = data_test['Facies']

facies_labels.describe()
# Store well labels and depths
well = data['Well Name'].values
depth = data['Depth'].values
well_test = data_test['Well Name'].values
depth_test = data_test['Depth'].values
# Fill 'PE' missing values with mean
from sklearn import preprocessing
imp = preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(feature_vectors)
feature_vectors = imp.transform(feature_vectors)
imp.fit(feature_vectors_test)
feature_vectors_test = imp.transform(feature_vectors_test)


In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_vectors)
scaled_features = scaler.transform(feature_vectors) #ndarray now. 
scaler_test = StandardScaler().fit(feature_vectors_test)
scaled_features_test = scaler.transform(feature_vectors_test) #ndarray now. 
#scaled_features.shape
#scaler = StandardScaler().fit(feature_vectors)
#scaled_features = scaler.transform(feature_vectors) #ndarray now. 

#df3 = pd.DataFrame(np.random.randn(10, 5),columns=['a', 'b', 'c', 'd', 'e'])
#df3.describe
scaled_features.shape

(2783, 7)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(scaled_features, facies_labels,test_size=0.05, random_state=42,stratify=facies_labels)
#X_train, X_cv, y_train, y_cv = train_test_split(scaled_features, facies_labels,test_size=0.05, random_state=42)
#y_train.value_counts()
X_train.shape

(2643, 7)

In [5]:
from sklearn.metrics import classification_report
target_names = ['SS', 'CSiS', 'FSiS', 'SiSh','MS', 'WS', 'D','PS', 'BS']

In [11]:
#from sklearn.neural_network import MLPClassifier
from xgboost.sklearn import  XGBClassifier
clf = XGBClassifier(learning_rate=0.12,max_depth=3,min_child_weight=10,n_estimators=150,seed=100,colsample_bytree=0.9)
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=0, learning_rate=0.12, max_delta_step=0,
       max_depth=3, min_child_weight=10, missing=None, n_estimators=150,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=100,
       silent=True, subsample=1)

In [12]:
y_cv_pred = clf.predict(X_cv) 
print(classification_report(y_cv, y_cv_pred,target_names=target_names))

             precision    recall  f1-score   support

         SS       0.83      0.62      0.71         8
       CSiS       0.70      0.70      0.70        33
       FSiS       0.63      0.76      0.69        25
       SiSh       0.75      0.33      0.46         9
         MS       0.62      0.50      0.56        10
         WS       0.65      0.55      0.59        20
          D       1.00      0.75      0.86         4
         PS       0.56      0.78      0.65        23
         BS       0.86      0.75      0.80         8

avg / total       0.68      0.66      0.66       140



C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
y_pred = clf.predict(scaled_features_test)

C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [10]:

print(classification_report(facies_labels_test, y_pred,target_names=target_names))

             precision    recall  f1-score   support

         SS       0.60      0.13      0.22        89
       CSiS       0.37      0.82      0.51        89
       FSiS       0.86      0.57      0.69       117
       SiSh       0.11      0.29      0.15         7
         MS       0.15      0.11      0.12        19
         WS       0.63      0.63      0.63        71
          D       0.73      0.65      0.69        17
         PS       0.63      0.55      0.59        40

avg / total       0.61      0.52      0.51       449



C:\Users\Homeuser\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1428: UserWarning: labels size, 8, does not match size of target_names, 9
  .format(len(labels), len(target_names))
